In [1]:
import os
import string
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import en_core_web_sm
import lightgbm as lgb

from sklearn import preprocessing, model_selection, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

nlp = en_core_web_sm.load()

pd.options.display.max_columns = 1000

In [4]:
!ls ../data/adhoc

train_labels.csv


In [5]:
train_df = pd.read_csv("../data/adhoc/train_labels.csv")
sub_df = pd.read_csv("../submissions/baseline.csv")

In [7]:
!ls ../data/train_data

adhoc  test_data  train_data


In [10]:
train_data_path = "../data/train_data/"
test_data_path = "../data/test_data/"

In [11]:
def get_currency(fname, act_currency, data_path):
    currency = "USD"
    with open(data_path + fname + ".xml") as file_name:
        xml_text = file_name.read()
        soup = BeautifulSoup(xml_text)
        
        for block in soup.find_all('block'):
            block_text = block.get_text().strip().lower()
            if "termination currency" in block_text:
                if "euro" in block_text:
                    currency = "EUR"
    return currency

# act_currency = []
# pred_currency = [] 
# for ind, row in train_df.iterrows():
#     fname = row["File_Name"]
#     currency =  row["TC_currency"] 
#     act_currency.append(currency)
#     pred_currency.append( get_currency(fname, currency, train_data_path) )
# train_df["pred_currency"] = pred_currency
# print((train_df["TC_currency"] == train_df["pred_currency"]).sum())

pred_currency = []
for ind, row in sub_df.iterrows():
    currency = "USD"
    fname = row["File_Name"]
    pred_currency.append( get_currency(fname, currency, test_data_path) )
sub_df["TC_currency"] = pred_currency

In [12]:
def get_data(fname, act, data_path):
    pred = "Party A"
    with open(data_path + fname + ".xml") as file_name:
        xml_text = file_name.read()
        soup = BeautifulSoup(xml_text)
        
        for block in soup.find_all('block'):
            block_text = block.get_text().strip().lower()
            if "calculation agent is party b" in block_text:
                pred = "Party B"
    return pred

# acts = []
# preds = []
# for ind, row in train_df.iterrows():
#     fname = row["File_Name"]
#     act = row["calulation_agent"]
#     acts.append(act)
#     pred = get_data(fname, act, train_data_path)
#     preds.append(pred)
# train_df["pred_calculation_agent"] = preds
# print((train_df["calulation_agent"] == train_df["pred_calculation_agent"]).sum())
# print((train_df["calulation_agent"] == "Party A").sum())

preds = []
for ind, row in sub_df.iterrows():
    act = "Party A"
    fname = row["File_Name"]
    pred = get_data(fname, act, test_data_path)
    preds.append(pred)
sub_df["calulation_agent"] = preds
sub_df["calulation_agent"].value_counts()

Party A    57
Party B     4
Name: calulation_agent, dtype: int64

In [13]:
sub_df.head()

,File_Name,date,party_A,party B,TC_currency,TC_Bespoke,calulation_agent,CA_fallback_default_dispute,CA_dispute_resolution,RD_method_party_A,RD_party_A_short_term_debt_classification,RD_party_A_moody_short_term_trigger_method,RD_party_A_moody_short_term_threshold,RD_party_A_SnP_short_term_trigger_method,RD_party_A_SnP_short_term_threshold,RD_party_A_Fitch_short_term_trigger_method,RD_party_A_Fitch_short_term_threshold,RD_party_A_long_term_debt_classification,RD_party_A_moody_long_term_trigger_method,RD_party_A_moody_long_term_threshold,RD_party_A_SnP_long_term_trigger_method,RD_party_A_SnP_long_term_threshold,RD_party_A_Fitch_long_term_trigger_method,RD_party_A_Fitch_long_term_threshold,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure,multibranch_party_A,multibranch_party_B,governing law
0,20_bk,NaN,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,No,1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,below,NotFound,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,worldwide,No,English Law
1,43_bk,NaN,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,No,1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,below,NotFound,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,worldwide,No,English Law
2,66_sh,NaN,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,No,1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,below,NotFound,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,worldwide,No,English Law
3,ch5,NaN,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,No,1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,below,NotFound,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,worldwide,No,English Law
4,76_sh,NaN,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,No,1,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,NotFound,below,NotFound,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,worldwide,No,English Law
